<a href="https://colab.research.google.com/github/j03hanafi/bigdata-unand/blob/main/pertemuan-9/latihan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A. Errors and Debugging**

## **1. Controlling Exceptions: %xmode**

In [2]:
def func1(a, b):
    return a / b
  
def func2(x):
    a = x
    b = x - 1
    return func1(a, b)

func2(1)

ZeroDivisionError: ignored


Memanggil func2 menghasilkan sebuah error, dengan magic function %xmode IPython bisa mengontrol jumlah informasi yang ditampilkan ketika error muncul. Ada tiga macam %xmode, plain, context, dan verbose.

In [3]:
%xmode Plain

Exception reporting mode: Plain


In [4]:
func2(1)

ZeroDivisionError: ignored

In [5]:
%xmode Verbose

Exception reporting mode: Verbose


In [6]:
func2(1)

ZeroDivisionError: ignored

## **2. Debugging: When Reading Tracebacks Is Not Enough**

In [7]:
%debug

> <ipython-input-2-88cab35a38d9>(2)func1()
      1 def func1(a, b):
----> 2     return a / b
      3 
      4 def func2(x):
      5     a = x

ipdb> 1
1
ipdb> a
a = 1
b = 0
ipdb> b
ipdb> quit



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



In [8]:
%debug

> <ipython-input-2-88cab35a38d9>(2)func1()
      1 def func1(a, b):
----> 2     return a / b
      3 
      4 def func2(x):
      5     a = x

ipdb> up
> <ipython-input-2-88cab35a38d9>(7)func2()
      5     a = x
      6     b = x - 1
----> 7     return func1(a, b)
      8 
      9 func2(1)

ipdb> print(x)
1
ipdb> up
> <ipython-input-6-7cb498ea7ed1>(1)<module>()
----> 1 func2(1)

ipdb> down
> <ipython-input-2-88cab35a38d9>(7)func2()
      5     a = x
      6     b = x - 1
----> 7     return func1(a, b)
      8 
      9 func2(1)

ipdb> quit


Ini memungkinkan user dengan cepat mengetahui penyebab kesalahan dan juga fungsi menyebabkan kesalahan. Lalu menggunakan magic function %pbd untuk mengaktifkan debugger secara otomatis ketika terjadi error.

In [9]:
%xmode Plain
%pdb on
func2(1)

Exception reporting mode: Plain
Automatic pdb calling has been turned ON


ZeroDivisionError: ignored

> <ipython-input-2-88cab35a38d9>(2)func1()
      1 def func1(a, b):
----> 2     return a / b
      3 
      4 def func2(x):
      5     a = x

ipdb> print(b)
0
ipdb> quit


# **B. Profiling and Timing Code**

## **1. Timing Code Snippets: %timeit and %time**

In [10]:
%timeit sum(range(100))

1000000 loops, best of 5: 1.08 µs per loop


%timeit menghitung waktu eksekusi berulang dari sebuah statement untuk akurasi yang lebih tepat.



In [11]:
%%timeit
total = 0
for i in range(1000):
  for j in range(1000):
    total += i * (-1) ** j

1 loop, best of 5: 333 ms per loop


%%timeit menghitung waktu eksekusi berulang dari kode.

In [12]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

The slowest run took 30.73 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 723 µs per loop



Menyortir list yang sudah disortir sebelumnya memakan waktu yang lebih cepat daripada menyortir list yang belum di sortir dengan perulangan. Fungsi magic %time merupakan pilihan yang tepat.

In [13]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
CPU times: user 20.7 ms, sys: 991 µs, total: 21.7 ms
Wall time: 21.7 ms


In [14]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
CPU times: user 2.98 ms, sys: 0 ns, total: 2.98 ms
Wall time: 3.14 ms


Berikut terlihat hasil perbedaan waktu ketika menyortir list yang sudah disortir dibandingkan dengan yang belum disortir. Terlihat juga bahwa fungsi magic %timeit memakan waktu yang lebih cepat dibandingkan %time.

Untuk %time dan %timeit, menggunakan double-percent-sign memungkinkan untuk melakukan timing multiline script.

In [15]:

%%time
total = 0
for i in range(1000):
  for j in range(1000):
    total += i * (-1) ** j

CPU times: user 406 ms, sys: 906 µs, total: 407 ms
Wall time: 407 ms


## **2. Profiling Full Scripts: %prun**

In [16]:
def sum_of_lists(N):
  total = 0
  for i in range(5):
    L = [j ^ (j >> i) for j in range(N)]
    total += sum(L)
  return total

# Memanggil %prun dengan function call untuk menampilkan deskripsi hasil

In [17]:
%prun sum_of_lists(1000000)

(hasil terlihat pada tab bantuan)
Pada hasil terlihat total waktu pada setiap function call, dimana eksekusi yang banyak memakan waktu.

## **3. Line-by-Line Profiling with %lprun**

In [18]:
pip install line_profiler

     |████████████████████████████████| 71kB 8.7MB/s 


In [21]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [22]:
%lprun -f sum_of_lists sum_of_lists(5000)
# Dengan %lprun, deskripsi dari proses dijelaskan per linenya.

## **4. Profiling Memory Use: %memit and %mprun**

In [23]:
pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp37-none-any.whl size=30180 sha256=f01bdfe3eb0bd678576b273d184b6723b1419e18634ff6db954c0298f2852217
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built memory-profiler


In [25]:

%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [26]:
%memit sum_of_lists(1000000)

peak memory: 211.00 MiB, increment: 56.80 MiB


Ekstensi memory profiler menyediakan dua fungsi magic, yaitu %memit (menyediakan perhitungan memori yang setara dengan %timeit) dan %mprun (menyediakan perhitungan memori yang setara dengan %lprun).

Pada sintaks diatas terlihat bahwa memori yang digunakan sebesar 211.00 MB.

In [27]:
%%file mprun_demo.py
def sum_of_lists(N):
  total = 0
  for i in range(5):
    L = [j ^ (j >> i) for j in range(N)]
    total += sum(L)
    del L # remove reference to L
  return total

Writing mprun_demo.py


In [ ]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(1000000)

(hasil terlihat pada tab bantuan)
Pada kolom increment terlihat seberapa besar setiap baris memengaruhi total memori.